In [1]:
# use subgraph 
# shared state schemas 
# different state schemas 

#### Shared State Schemas

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, MessagesState
from langgraph.constants import START, END

def subgraph_node_1(state: MessagesState)-> MessagesState:
    return {"messages":"subgraph node 1 is execute"}

subgraph = (
    StateGraph(MessagesState)
    .add_node("subgraph_node_1", subgraph_node_1)
    .set_entry_point("subgraph_node_1")
    .compile()
)

graph = (
    StateGraph(MessagesState)
    .add_node("subgraph", subgraph)
    .set_entry_point("subgraph")
    .add_edge("subgraph", END)
    .compile()
)

In [4]:
graph.invoke({"messages": "hello"})

{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='f5e07331-4e73-42b3-94fa-5b15503b1e42'),
  HumanMessage(content='subgraph node 1 is execute', additional_kwargs={}, response_metadata={}, id='13e12cad-1c9d-40fa-832c-0a681377f723')]}

#### Different state schemas

In [8]:
from langgraph.graph import StateGraph
from langgraph.constants import START, END
from typing import TypedDict

class SubGraphState(TypedDict):
    subgraph_messages: str

class GraphState(TypedDict):
    messages: str

def subgraph_node_1(state: SubGraphState)-> SubGraphState:
    return {"subgraph_messages": state["subgraph_messages"]}

subgraph = (
    StateGraph(SubGraphState)
    .add_node("subgraph_node_1", subgraph_node_1)
    .set_entry_point("subgraph_node_1")
    .compile()
)

def graph_node_1(state: GraphState)-> GraphState:
    response = subgraph.invoke({"subgraph_messages": f"subgraph is execute, {state['messages']}"})
    return {"messages": response["subgraph_messages"]}

graph = (
    StateGraph(GraphState)
    .add_node("graph_node_1", graph_node_1)
    .set_entry_point("graph_node_1")
    .add_edge("graph_node_1", END)
    .compile()
)



In [9]:
graph.invoke({"messages":"space"})

{'messages': 'subgraph is execute, space'}

#### Add persistence 

In [11]:
# add persistence for subgraph
# subgraph_builder = StateGraph(MessagesState)
# subgraph = subgraph_builder.compile(checkpointer=True)
# subgraph

#### View subgraph state

In [ ]:
# graph.get_state(config)
# graph.get_state(config, subgraph=True)

#### Stream subgraph outputs

In [12]:
# for chunk in workflow.stream({"messages":"demo"}, subgraph=True, stream_mode="updates"):
#     print(chunk)